In [1]:
import pandas as pd
customer = pd.read_csv("customer_join.csv")
uselog_months = pd.read_csv("use_log_months.csv")

In [2]:
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [3]:
uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [4]:
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()

In [5]:
for i in range(1,len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"},inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"},inplace=True)
    tmp = pd.merge(tmp,tmp_before,on="customer_id",how="left")
    uselog = pd.concat([uselog,tmp],ignore_index=True)
uselog.head()

C:\Users\zaxir\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [6]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

<ipython-input-6-07e0fc284be4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["exit_date"] = None
<ipython-input-6-07e0fc284be4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])


In [7]:
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
    
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog,exit_customer,on=["customer_id","年月"],how="left")
print(len(uselog))
exit_uselog.head()

C:\Users\zaxir\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-7-04f9944db8e3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)


33851


<ipython-input-7-04f9944db8e3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
exit_customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,年月
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00,201803
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00,201803
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23,2018-03-30 00:00:00,201803
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23,2018-04-30 00:00:00,201804
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23,2018-04-30 00:00:00,201804


In [11]:
uselog.dtypes

年月              object
customer_id     object
count_0          int64
count_1        float64
dtype: object

In [12]:
exit_customer.dtypes

customer_id                  object
name                         object
class                        object
gender                       object
start_date                   object
end_date             datetime64[ns]
campaign_id                  object
is_deleted                    int64
class_name                   object
price                         int64
campaign_name                object
mean                        float64
median                      float64
max                           int64
min                           int64
routine_flg                   int64
calc_date                    object
membership_period             int64
exit_date                    object
年月                           object
dtype: object

In [13]:
exit_uselog = pd.merge(uselog,exit_customer,on=["customer_id","年月"],how="left")

In [14]:
exit_uselog.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


In [16]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog,conti_customer,on=["customer_id"],how="left")
print(len(conti_uselog))

33851


In [17]:
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

27422


In [18]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201812,GD648354,4,8.0,XXXXX,C01,M,2016-11-01,NaN,CA1,...,オールタイム,10500.0,通常,5.083333,5.0,8.0,2.0,1.0,2019-04-30,29.0
1,201901,OA576440,3,5.0,XXXXX,C01,F,2016-09-01,NaN,CA1,...,オールタイム,10500.0,通常,4.666667,4.5,7.0,3.0,1.0,2019-04-30,31.0
2,201807,IK248737,3,4.0,XXXXXX,C02,F,2015-08-01,NaN,CA1,...,デイタイム,7500.0,通常,4.000000,4.0,6.0,2.0,1.0,2019-04-30,44.0
3,201806,PL746077,7,7.0,XXXXX,C01,M,2016-10-01,NaN,CA1,...,オールタイム,10500.0,通常,4.916667,5.0,7.0,2.0,1.0,2019-04-30,30.0
4,201806,HD858116,6,4.0,XXXX,C01,F,2016-09-01,NaN,CA1,...,オールタイム,10500.0,通常,5.083333,4.5,9.0,3.0,1.0,2019-04-30,31.0


In [19]:
predict_data = pd.concat([conti_uselog,exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201812,GD648354,4,8.0,XXXXX,C01,M,2016-11-01,NaN,CA1,...,10500.0,通常,5.083333,5.0,8.0,2.0,1.0,2019-04-30,29.0,NaN
1,201901,OA576440,3,5.0,XXXXX,C01,F,2016-09-01,NaN,CA1,...,10500.0,通常,4.666667,4.5,7.0,3.0,1.0,2019-04-30,31.0,NaN
2,201807,IK248737,3,4.0,XXXXXX,C02,F,2015-08-01,NaN,CA1,...,7500.0,通常,4.000000,4.0,6.0,2.0,1.0,2019-04-30,44.0,NaN
3,201806,PL746077,7,7.0,XXXXX,C01,M,2016-10-01,NaN,CA1,...,10500.0,通常,4.916667,5.0,7.0,2.0,1.0,2019-04-30,30.0,NaN
4,201806,HD858116,6,4.0,XXXX,C01,F,2016-09-01,NaN,CA1,...,10500.0,通常,5.083333,4.5,9.0,3.0,1.0,2019-04-30,31.0,NaN


In [20]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"],format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])

In [28]:
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i],predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)

<ipython-input-28-0eb3cc08c319>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data["period"][i] = int(delta.years*12 + delta.months)


In [29]:
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201812,GD648354,4,8.0,XXXXX,C01,M,2016-11-01,NaN,CA1,...,5.083333,5.0,8.0,2.0,1.0,2019-04-30,29.0,NaN,25,2018-12-01
1,201901,OA576440,3,5.0,XXXXX,C01,F,2016-09-01,NaN,CA1,...,4.666667,4.5,7.0,3.0,1.0,2019-04-30,31.0,NaN,28,2019-01-01
2,201807,IK248737,3,4.0,XXXXXX,C02,F,2015-08-01,NaN,CA1,...,4.000000,4.0,6.0,2.0,1.0,2019-04-30,44.0,NaN,35,2018-07-01
3,201806,PL746077,7,7.0,XXXXX,C01,M,2016-10-01,NaN,CA1,...,4.916667,5.0,7.0,2.0,1.0,2019-04-30,30.0,NaN,20,2018-06-01
4,201806,HD858116,6,4.0,XXXX,C01,F,2016-09-01,NaN,CA1,...,5.083333,4.5,9.0,3.0,1.0,2019-04-30,31.0,NaN,21,2018-06-01


In [30]:
predict_data.isnull().sum()

年月                      0
customer_id             0
count_0                 0
count_1               258
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [31]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               258
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [32]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2636
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2636
period                  0
now_date                0
dtype: int64

In [33]:
target_col = ["campaign_name","class_name","gender","count_1","routine_flg","period","is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,オールタイム,M,8.0,1.0,25,0.0
1,通常,オールタイム,F,5.0,1.0,28,0.0
2,通常,デイタイム,F,4.0,1.0,35,0.0
3,通常,オールタイム,M,7.0,1.0,20,0.0
4,通常,オールタイム,F,4.0,1.0,21,0.0


In [34]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,8.0,1.0,25,0.0,0,0,1,1,0,0,0,1
1,5.0,1.0,28,0.0,0,0,1,1,0,0,1,0
2,4.0,1.0,35,0.0,0,0,1,0,1,0,1,0
3,7.0,1.0,20,0.0,0,0,1,1,0,0,0,1
4,4.0,1.0,21,0.0,0,0,1,1,0,0,1,0


In [35]:
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,8.0,1.0,25,0.0,0,0,1,0,0
1,5.0,1.0,28,0.0,0,0,1,0,1
2,4.0,1.0,35,0.0,0,0,0,1,1
3,7.0,1.0,20,0.0,0,0,1,0,0
4,4.0,1.0,21,0.0,0,0,1,0,1


In [36]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

In [37]:
exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

In [38]:
X = pd.concat([exit,conti],ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]

In [39]:
X_train,X_test,y_train,y_test = sklearn.model_selection.train_test_split(X,y)

In [40]:
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train,y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1.
 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0.
 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1.
 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0.
 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0.
 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1.
 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1.

In [41]:
results_test = pd.DataFrame({"y_test":y_test,"y_pred":y_test_pred})
results_test.head()

,y_test,y_pred
1565,0.0,0.0
1882,0.0,0.0
1034,1.0,0.0
536,1.0,1.0
1709,0.0,0.0


In [43]:
correct = len(results_test.loc[results_test["y_test"] == results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.8916349809885932


In [44]:
print(model.score(X_test,y_test))
print(model.score(X_train,y_train))

0.8916349809885932
0.9797211660329531
